# Running a QE calculation with AiiDA


After we have created an AiiDA profile and covered the most important concepts, let's now have a look at how we can
actually run an atomistic simulation. For this tutorial, we'll be using Quantum ESPRESSO (QE), but plugins for most other,
commonly used codes exist, as well. At the end of this notebook, we will show you how you can quickly get started with
those.


For a quantum mechanical simulation, we'll need various ingredients:
- The code we want to execute for which the necessary infrastructure is provided by the corresponding AiiDA plugin
- The atomic structure that should be calculated
- The calculation parameters, such as the plane-wave cutoff and convergence criteria, given by a dictionary
- The k-point sampling information
- The pseudopotentials

Each of these inputs is reperesented by an AiiDA class that inherits from the `Node` base class and takes care of
the database storage, and the recording of provenance. A visualization is provided in the figure below:

![](https://aiida-tutorials.readthedocs.io/en/latest/_images/Si_example.png)
<!-- # {height=150px align=center} -->

From the calculation (or `CalcJob` in AiiDA jargon), we obtain as outputs:
- The remote folder where the calculation was actually executed (if desired, this can automatically be cleaned after
  successful execution of a workflow)
- The retrieved files which are stored in AiiDA's file repository (these are the main output files, but not large
  intermediate files, e.g. charge density)
- The output parameters, which are specific entities that are being parsed from the output files, and thus actually
  stored in AiiDA's SQL databased, and are therefore also queryabel (more on that later)

Additional (parsed) outputs can further be defined by the plugin developer.

In the following, we will see how we can prepare all necessary inputs for the calculation, run it, and interact with it.
First, let's use the AiiDA jupyter magic, which will also load our default profile `fair-workflows`.

In [ ]:
%load_ext aiida
%aiida

---
We then import the two main modules needed for this example, the **Object-relational mapper** (ORM), which links
entities in AiiDA's SQL database to the Python objects we will be dealing with, as well as the engine necessary to
execute our calculation.

In [ ]:
from aiida import orm, engine

## `Computer`s and `Code`s

The `verdi presto` command used to create the AiiDA profile automatically sets up your local workstation as
the `localhost` computer. This will suffice for the sake of the tutorial, but to set up additional `Computer`s such as
remote HPC resources in the future, we point you to the [relevant section of the documentation on running external codes](https://aiida.readthedocs.io/projects/aiida-core/en/v2.6.1/howto/run_codes.html#how-to-set-up-a-computer).

We then create a `Code` instance for the `pw.x` executable of Quantum ESPRESSO. To keep the notebook idempotent, we are
using the Python API here, and try to load it if it already exists, otherwise we create it:

In [ ]:
from aiida.common.exceptions import NotExistent

try:
    pw_code = orm.load_code('qe-7.1-pw@localhost')  # The computer label can also be omitted here
    print("Loaded pw.x Code of QE v7.1.")
except NotExistent:
    pw_code = orm.InstalledCode(
        computer=orm.load_computer('localhost'),
        filepath_executable='/apps/share64/debian10/espresso/espresso-7.1/bin/pw.x',
        label='qe-7.1-pw',
        description='Serial version of QE 7.1 pw.x on localhost',
        default_calc_job_plugin='quantumespresso.pw',
        prepend_text='export OMP_NUM_THREADS=1',
        append_text='',
        use_double_quotes=False,
        with_mpi=False
    ).store()
    print("Created and stored pw.x Code of QE v7.1.")


To create a `Code` in AiiDA, various settings are required:
- First, the `Computer` where the code should be executed needs to be specified
- The absolute path to the executable must be given, as well, and we have already added the correct path for the nanoHUB deployment
- A label (to load the `Code` later on), and a description (optional) are also given
- As mentioned above, the way how AiiDA interacts with a given executable, e.g. which input files and parameters are
  required, is given by the AiiDA plugins, not `aiida-core` itself. The `aiida-quantumespresso` plugin is already
  installed in this workspace, so we can directly set `quantumespresso.pw` as the `default_calc_job_plugin`
- In addition, `append_text` and `prepend_text` can be added, and will appear in the submission script before and after
  the actual call to the executable. This can be useful to load modules or set environment variables (as done here to
  disable hyperthreading)
- As the version of QE available on the `localhost` `Computer` of the nanoHUB deployment is serial, we disable MPI via `with_mpi=False`

We can further check which `Calculation` plugins are available by running:

In [ ]:
%verdi plugin list aiida.calculations

Note that AiiDA's `verdi` command-line interface (CLI) can also be (and typically is) used to set up a `Code` instance for a profile. To this end, the command:

```shell
verdi code create core.code.installed
```

needs to be run on the terminal and will ask you for all required options.

For convenience, it is also possible to provide these options via a YAML configuration file using the `--config` flat,
which can point either to a local file, or to a URL (e.g. on GitHub). The necessary YAML configuration file to set up
`pw.x` of QE v7.1 is part of this repository.

In addition, the [aiida-code-registry](https://github.com/aiidateam/aiida-code-registry/) compiles `Computer` and `Code`
configuration files of commonly used HPC resources. If you don't find your HPC or code there, pull requests are very
welcome.

After creating our `Code`, we can then see if everything works fine by running:

In [ ]:
%verdi code test qe-7.1-pw

## The atomic structure

Next, we need to create the atomic structure for which we actually want to run our simulation. For this, we are using
the atomic simulation environment (ASE) and pass it to the `StructureData` AiiDA class:

In [ ]:
from ase.build import bulk
si_structure = orm.StructureData(ase=bulk('Si', 'diamond', a=5.43))
si_structure

We can see that the `StructureData` has been created and a UUID assigned to it, but it is still unstored. AiiDA will
automatically store it, as it becomes part of the provenance graph when we run the simulation. Alternatively, we could
store it manually here, note the PK or UUID, and reload it, such that we don't create a new structure every time we run
the notebook.

For high-throughput studies, one would often like to obtain the input structures from databases, rather than creating
them manually, as we did here. Naturally, AiiDA provides the necessary tools to do just that, and we point here to [some
of the available documentation](https://aiida-qe-demo.readthedocs.io/en/latest/4_generating_inputs.html#importing-structures-from-external-databases).

## The ProcessBuilder

Before we construct the remaining necessary inputs for running our calculation, let's take a minute to
actually see where they will end up. Eventually, they will be used to populate an AiiDA `CalcJob`, which is the main
class for running (remote) executables.

The `Code` instance, which has the necessary AiiDA plugin required to run the simulation attached, `quantumespresso.pw`
in our case,  provides the `get_builder` method, that returns a `ProcessBuilder` that helps us to attach our input
entities to the calculation. Effectively, this can be achieved as follows:

In [ ]:
builder = pw_code.get_builder()

# We attach the `Code` to the `ProcessBuilder`
builder.code = pw_code

# We attach the `StructureData` to the `ProcessBuilder`
builder.structure = si_structure

# Let's check out the `ProcessBuilder`
print(builder)
builder

As we can see above, the **Process class** of the `ProcessBuilder` has been correctly assigned to the `PwCalculation` as
defined in the `aiida-quantumespresso` plugin.

The `Code` and `Structure` is attached, but the `pseudos`, `metadata` and our calculation parameters are still missing,
so let's add them now!

## Pseudopotentials

The `aiida-pseudo` package provides a simple interface to download and install pseudopotentials into your AiiDA instance.
To install the PBE and PBEsol pseudos from the [*Standard solid-state pseudopotentials (SSSP)*
library](https://www.materialscloud.org/discover/sssp/table/efficiency) at *efficiency* precision, we just need to run:

In [ ]:
!/apps/share64/debian10/anaconda/anaconda-7/envs/AIIDA/bin/aiida-pseudo install sssp -x PBE -p efficiency
!/apps/share64/debian10/anaconda/anaconda-7/envs/AIIDA/bin/aiida-pseudo install sssp -x PBEsol -p efficiency

While we are using the full path to the `aiida-pseudo` binary here, if you have `aiida-pseudo` installed in your
`conda` or virtual environment and it activated, you should have the command directly available from the command line.

We can now verify that we have more `Node`s and `Group`s (corresponding to the two pseudopotential familias) in our
database via:

In [ ]:
%verdi storage info

Finally, we can attach the pseudopotentials needed to calculate our structure to the builder:

In [ ]:
pseudo_family = orm.load_group('SSSP/1.3/PBEsol/efficiency')
builder.pseudos = pseudo_family.get_pseudos(structure=si_structure)

## The calculation `parameters`

Of course, we also need to specify the typical settings for our calculation, such as the plane-wave cutoff, convergence
criteria, etc. These are recorded in the `parameters` attribute of the `ProcessBuilder`, and are given by a nested
dictionary (for users accustomed to Quantum ESPRESSO, the dictionary should look familiar):

In [ ]:
# Request the recommended wavefunction and charge density cutoffs
# for the given structure and energy units.
cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=si_structure,
    unit='Ry'
)

parameters = {
    'CONTROL': {
        'calculation': 'scf'
    },
    'SYSTEM': {
        'ecutwfc': cutoff_wfc,
        'ecutrho': cutoff_rho,
    }
}
builder.parameters = orm.Dict(parameters)

To keep things simple, we are only running a single self-consistent field (SCF) calculation here, so a rather small
parameter dictionary is sufficient to define all inputs. For larger workflows, possibly including multiple QE
calculations, the number of parameters to be specified can of course grow larger.

In addition to the inputs required for running QE, other settings, for instance to specify the requested resources on
the HPC must be provided. This would usually be done via the submission script, but AiiDA creates the submission script
for you on the remote machine, if the `Computer` has been set up correctly. These options are stored under `builder.metadata.options`:

In [ ]:
builder.metadata.options = {
    'resources': {
        'num_machines': 1,
    },
    'max_wallclock_seconds': 1800,
    # 'withmpi': False,
}

## K-points

The last missing ingredient now is the k-point sampling, which can be created either by providing the k-point spacing,
or an explicit mesh, as follows:

In [ ]:
# Generate a 2x2x2 Monkhorst-Pack mesh
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([2, 2, 2])
builder.kpoints = kpoints

## Submitting our calculation

Finally, we can run our calculation using the AiiDA engine:

In [ ]:
results, node = engine.run.get_node(builder)
print(f'Calculation: {node.process_class}<{node.pk}> {node.process_state.value} [{node.exit_status}]')
print(f'Results: {results}')

As our calculation is running and after it has finished, we can interact with it using the `verdi` CLI command, or the
`%verdi` jupyter magic command, e.g. via:

In [ ]:
# Check all processes executed in the last 24 hours
%verdi process list -ap 1

In [ ]:
# Check the status of the `PwCalculation` we just ran -> Should be `Finished [0]` hopefully
%verdi process status {node.pk}

In [ ]:
# Show more information, e.g. the inputs/outputs of the `PwCalculation` we just ran
%verdi process show {node.pk}

There are many more useful verdi commands to interact with your simulations, so we would like to point you to the [AiiDA
cheat sheet](https://aiida.readthedocs.io/projects/aiida-core/en/stable/reference/cheatsheet.html) that provides a
concise overview.

***

## Using protocols

In the above section, we have fully populated all necessary entries for the `ProcessBuilder` explicitly.
In addition to the `get_builder` method of the `Code` as we used before, many workflows (**note**, full workflows, not
individual calculations as we ran above) defined in certain AiiDA plugins also provide a `get_builder_from_protocol`
method, which provides a quicker way to set up the simulation.

Here, in addition to the `code` and `structure`, one of the entries *fast*, *moderate*, or *accurate* can be chosen as
the **protocol**, and calculation parameters are automatically set under the hood. This reduces the code to obtain a
fully populated builder to:

In [ ]:
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain

builder_protocol = PwBaseWorkChain.get_builder_from_protocol(
    code=pw_code, 
    structure=si_structure,
    protocol="fast",
)
builder_protocol

As you can see, various properties of the builder, and especially the calculation `parameters` have automatically been
populated for us. This not only provides a quicker way to get started with workflows in AiiDA, but also allows one to
start using new codes immediately, without having to learn their input tags first. Instead, one can just select a
protocol, and then look at the populated builder or the generated input files.

However, it should be noted that the `get_builder_from_protocol` method is not implemented in `aiida-core` but in the
plugin, and might therefore not be available for the AiiDA plugin you might want to use. More specifically, it
originated from the AiiDA common workflows project (the publication can be found
[here](https://www.nature.com/articles/s41524-021-00594-6)), which aimed to provide a common interface for computing
material properties using different quantum engines:

![](https://www.aiida.net/_images/aiida-common-wf-image-2000x1000-1.png)

The settings specified in the pre-defined protocols can be overwritten, using nested `parameter` dictionaries,
direct assignment to the builder attributes, or YAML files (more in the [relevant documentation
section](https://aiida-tutorials.readthedocs.io/en/latest/sections/running_processes/workflows.html#submitting-a-work-chain)).


Finally, we can run the workflow, just as we did above:

In [ ]:
results, node = engine.run.get_node(builder_protocol)
print(f'Calculation: {node.process_class}<{node.pk}> {node.process_state.value} [{node.exit_status}]')
print(f'Results: {results}')

Finally, don't hesitate to explore the results of your workflow, `%verdi` is your friend ;)

In [ ]:
%verdi process show {node.pk}